<a href="https://colab.research.google.com/github/MaxVargas1994/GAN/blob/master/MejorarCalidadDeVideo_Metodo2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Configurado por el canal de YouTube de TICnoticos.
Enlace: https://www.youtube.com/@Ticnoticos


# Bajar peso de video rápido y fácil:
Enlace: https://www.youtube.com/watch?v=0ex7DyTrHIQ


# Pasos e instrucciones:

# **1.** Clonación del proyecto que contiene la IA y otras dependencias

In [1]:
%cd /content
!rm -rf GFPGAN
!git clone https://github.com/TencentARC/GFPGAN.git
%cd GFPGAN

!pip install basicsr
!pip install facexlib
!pip install -r requirements.txt
!python setup.py develop
!pip install realesrgan
# Download the pre-trained model
!wget https://github.com/TencentARC/GFPGAN/releases/download/v0.2.0/GFPGANCleanv1-NoCE-C2.pth -P experiments/pretrained_models

/content
Cloning into 'GFPGAN'...
remote: Enumerating objects: 523, done.
remote: Total 523 (delta 0), reused 0 (delta 0), pack-reused 523
Receiving objects: 100% (523/523), 5.39 MiB | 4.40 MiB/s, done.
Resolving deltas: 100% (264/264), done.
/content/GFPGAN
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 172.5/172.5 kB 3.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 299.2/299.2 kB 22.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 76.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 254.7/254.7 kB 33.1 MB/s eta 0:00:00
  Created wheel for basicsr: filename=basicsr-1.4.2-py3-none-any.whl size=214818 sha256=810c2cbf002cd894d87ac113406fd919d2f1f9a7473427f961aa215c6231e0d3
  Stored in directory: /root/.cache/pip/wheels/38/83/99/2d8437cc652a01af27df5ff037a4075e95b52d67705c5f30ca
Successfully built basicsr
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.6/59.6 kB 1.6 MB/s eta 0:00:00
     ━━━━━━

# 2. Importación y estructuración de carpetas

In [2]:
import os
from google.colab import files
import shutil

upload_folder = 'upload'
result_folder = 'results'
video_folder = 'videos'
video_result_folder = 'results_videos'
video_mp4_result_folder = 'results_mp4_videos'

if os.path.isdir(upload_folder):
  print(upload_folder+" exists")
else :
  os.mkdir(upload_folder)

if os.path.isdir(video_folder):
  print(video_folder+" exists")
else :
  os.mkdir(video_folder)

if os.path.isdir(video_result_folder):
  print(video_result_folder+" exists")
else :
  os.mkdir(video_result_folder)

if os.path.isdir(video_mp4_result_folder):
  print(video_mp4_result_folder+" exists")
else :
  os.mkdir(video_mp4_result_folder)

if os.path.isdir(result_folder):
  print(result_folder+" exists")

else :
  os.mkdir(result_folder)




# 3. Procesamiento y mejora de calidad

In [3]:
import cv2
import numpy as np
import glob
from os.path import isfile, join
import subprocess
from IPython.display import clear_output


directory = '/content/GFPGAN/videos'
zee = 0

for f in os.listdir(upload_folder):
    os.remove(os.path.join(upload_folder, f))

#clearing previous .avi files
for f in os.listdir(video_result_folder):
    os.remove(os.path.join(video_result_folder, f))

#clearing .mp4 result files
for f in os.listdir(video_mp4_result_folder):
    os.remove(os.path.join(video_mp4_result_folder, f))


def convert_frames_to_video(pathIn,pathOut,fps):
    frame_array = []
    files = [f for f in os.listdir(pathIn) if isfile(join(pathIn, f))]
    #for sorting the file names properly
    files.sort(key = lambda x: int(x[5:-4]))
    size2 = (0,0)

    for i in range(len(files)):
        filename=pathIn + files[i]
        img = cv2.imread(filename)
        height, width, layers = img.shape
        size = (width,height)
        size2 = size
        print(filename)
        frame_array.append(img)
    out = cv2.VideoWriter(pathOut,cv2.VideoWriter_fourcc(*'DIVX'), fps, size2)
    for i in range(len(frame_array)):
        out.write(frame_array[i])
    out.release()

for filename in os.listdir(directory):
    f = os.path.join(directory, filename)
    # checking if it is a file
    if os.path.isfile(f):

      print("PROCESSING :"+str(f)+"\n")
      #video to frames
      cam = cv2.VideoCapture(str(f))
      try:
          if not os.path.exists('/content/GFPGAN/upload'):
              os.makedirs('/content/GFPGAN/upload')
      except OSError:
          print ('Error: Creating directory of data')

      # frame
      currentframe = 0

      while(True):
          ret,frame = cam.read()

          if ret:
              name = '/content/GFPGAN/upload/frame' + str(currentframe) + '.jpg'
              cv2.imwrite(name, frame)
              currentframe += 1
              print(currentframe)
          else:

              break

      cam.release()
      cv2.destroyAllWindows()

      !python inference_gfpgan.py -i upload -o results -v 1.2 -s 2 --bg_upsampler realesrgan

      #after upscaling just delete the source frames
      for f in os.listdir(upload_folder):
          os.remove(os.path.join(upload_folder, f))

      '''
      #rename all frames in "results" to remove the 'out' substring from the processing results
      paths = (os.path.join(root, filename)
              for root, _, filenames in os.walk('/content/GFPGAN/results')
              for filename in filenames)
      for path in paths:
          newname = path.replace('_out', '')
          if newname != path:
              os.rename(path, newname)
      '''

      #convert super res frames to .avi
      pathIn = '/content/GFPGAN/results/restored_imgs/'

      zee = zee+1
      fName = "video"+str(zee)
      filenameVid = f"{fName}.avi"

      pathOut = "/content/GFPGAN/results_videos/"+filenameVid

      fps = 29.97 #change this to FPS of your source video

      convert_frames_to_video(pathIn, pathOut, fps)

      #after processing frames converted to .avi video , delete upscaled frames from previous video
      for f in os.listdir('results/restored_imgs'):
          os.remove(os.path.join('results/restored_imgs', f))

      #convert .avi to .mp4
      src = '/content/GFPGAN/results_videos/'
      dst = '/content/GFPGAN/results_mp4_videos/'

      for root, dirs, filenames in os.walk(src, topdown=False):
          #print(filenames)
          for filename in filenames:
              print('[INFO] 1',filename)
              try:
                  _format = ''
                  if ".flv" in filename.lower():
                      _format=".flv"
                  if ".mp4" in filename.lower():
                      _format=".mp4"
                  if ".avi" in filename.lower():
                      _format=".avi"
                  if ".mov" in filename.lower():
                      _format=".mov"

                  inputfile = os.path.join(root, filename)
                  print('[INFO] 1',inputfile)
                  outputfile = os.path.join(dst, filename.lower().replace(_format, ".mp4"))
                  subprocess.call(['ffmpeg', '-i', inputfile, outputfile])
              except:
                  print("An exception occurred")

      clear_output(wait=True)

      #clearing previous .avi files
      for f in os.listdir(video_result_folder):
          os.remove(os.path.join(video_result_folder, f))

Se han truncado las últimas 5000 líneas del flujo de salida.
/content/GFPGAN/results/restored_imgs/frame193.jpg
/content/GFPGAN/results/restored_imgs/frame194.jpg
/content/GFPGAN/results/restored_imgs/frame195.jpg
/content/GFPGAN/results/restored_imgs/frame196.jpg
/content/GFPGAN/results/restored_imgs/frame197.jpg
/content/GFPGAN/results/restored_imgs/frame198.jpg
/content/GFPGAN/results/restored_imgs/frame199.jpg
/content/GFPGAN/results/restored_imgs/frame200.jpg
/content/GFPGAN/results/restored_imgs/frame201.jpg
/content/GFPGAN/results/restored_imgs/frame202.jpg
/content/GFPGAN/results/restored_imgs/frame203.jpg
/content/GFPGAN/results/restored_imgs/frame204.jpg
/content/GFPGAN/results/restored_imgs/frame205.jpg
/content/GFPGAN/results/restored_imgs/frame206.jpg
/content/GFPGAN/results/restored_imgs/frame207.jpg
/content/GFPGAN/results/restored_imgs/frame208.jpg
/content/GFPGAN/results/restored_imgs/frame209.jpg
/content/GFPGAN/results/restored_imgs/frame210.jpg
/content/GFPGAN/resul